In [11]:
import os
import sys
import requests
from cachetools import cached, TTLCache
import time
import pandas as pd
import tqdm

g_token=''

@cached(cache=TTLCache(maxsize=1, ttl=36000))
def get_token():
    # step 1:  using username='30007', password='2A4nFXhP' 
    ##         on https://mid.meddra.org/account/register for apikey
    # step 2:  apply for grant
    cmd = '''
    curl \
        --request POST \
        --data "grant_type=password"  \
        --data "username={username}"  --data "password={apikey}" \
        --data "scope=meddraapi" \
        --data "client_id=mspclient" \
        "https://mid.meddra.org/connect/token"
    '''.format(username='30007', apikey='4Lj68iPSy3jjti7hK2hM')
    os.system(cmd)
    # ref: https://cloud.tencent.com/developer/article/1908468
    return auth['access_token']
    # {"access_token":"6acb65aaaa2eee65530de8e2521606b10051a5b8a5451bdf1227eb704bda080d","expires_in":36000,"token_type":"Bearer"}

def get_token2():
    return "0ee4891cc555c08327623cf1e37697aca59bba27744b0006e138d2f0e6a47fc7"

def get_soc(row):
    ## (1) search pt name for pt-code
    headers = {
        'accept': 'application/json',
        'Authorization': 'Bearer {}'.format(get_token2()),
        'Content-Type': 'text/json',
    }

    pt_name = row['pt']   # "heart valve operation"

    data = '{"bview":"SOC","rsview":"release","language":"English","stype":1,"addlangs":[],"filters":[],"version":24,"searchterms":[{"searchtype":0,"searchterm":"' \
           + pt_name + '","searchlogic":0}],"kana":false,"idiacritical":true,"synonym":false,"contains":false,"soc":false,"hlgt":false,"hlt":false,"pt":true,"llt":false,"smq":false,"skip":0,"take":0,"separator":2}'
    
    try:
        response = requests.post('https://gxpmapiprod.meddra.org/api/search', headers=headers, data=data)
        #print(response.json())
        ##(2) detail: search soc name & code using pt_code
        pt_code = response.json()[0]['pcode']
    
        data = '{"bview":"SOC","rsview":"release","code":' + str(pt_code) + \
               ',"pcode":0,"syncode":0,"lltcode":0,"ptcode":0,"hltcode":0,"hlgtcode":0,"soccode":0,"smqcode":0,"type":"PT","addlangs":[],"rtype":"P","lang":"English","ver":24,"kana":false,"separator":2}'
    
        response = requests.post('https://gxpmapiprod.meddra.org/api/detail', headers=headers, data=data)
        soc_name = ''
        soc_code = -1
    
        for item in response.json()['mds']:
            #print(item)
            if item['primarysocfg'] == 'Y':
                soc_name = item['socname']
                soc_code = item['soccode']
                           
        ## finally output results using dataframes
        #return {'pt': pt_name, 'pt_code': pt_code, 'soc_name': soc_name, 'soc_code': soc_code}
        return pd.Series([pt_name, pt_code, soc_name, soc_code], index=['pt_name', 'pt_code', 'soc_name', 'soc_code'])

    except Exception as e:
        print("error for: " + pt_name)
    finally:
        pass


In [ ]:
conn = pymysql.connect(
        host='localhost',
        port=int(3306),
        user="root",
        passwd='0921051011Liu',
        db="faers",
        charset='utf8mb4')

# 去除异常字符
sql = '''   
   SELECT 
        replace(coalesce(t2.pt,'not_found_pt'),'\\r', '') as pt,
        COUNT(distinct t1.CASEID) AS caseid_cnt,
        COUNT(distinct t1.PRIMARYID) AS primaryid_cnt
    FROM faers.ribo_drug_usage t1
    left join faers.faers_reac t2
    on t1.primaryid = t2.primaryid and lower(t1.role_cod) = 'ps'
    group by 1
    order by 1;
'''
df_pt_stat = pd.read_sql_query(sql, conn)

conn.close()

In [ ]:
import tqdm
from tqdm import tqdm
tqdm.pandas(desc='apply')

df_pt_stat=pd.read_csv('./ribo_pt.csv')
## pt_soc_mapping = df_pt_stat.progress_apply(get_soc, axis=1)
pt_soc_mapping = df_pt_stat.apply(get_soc, axis=1)

#df_pt_stat[ df_pt_stat['caseid_cnt'] != df_pt_stat['primaryid_cnt']]